Importing necessary libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy import optimize
import pandas as pd
import glob
import math
import os

import multiprocessing
from multiprocessing import Pool

Importing dataset

In [2]:
# Specifying simulation directory and the directory to save results in
wdir = '/Users/thepoetoftwilight/Documents/CASSI2020/CASSI2020-Code+Results/Results/m10q_res250/'

# Specifying a snapshot for temporal analysis
sdir = wdir + 'temporal_analysis/'

# Specifying where to save plots
savepath = '/Users/thepoetoftwilight/Documents/CASSI2020/CASSI2020-Animations_Background_Images/poster_plots/'

In [3]:
def plot_sim_partition(sim_partition):
    
    for i in range(0, len(sim_partition)):
        
        snap = sim_partition[i][0]
        redshift = sim_partition[i][1]
        x_min = sim_partition[i][2]
        x_max = sim_partition[i][3]
        y_min = sim_partition[i][4]
        y_max = sim_partition[i][5]
        m = sim_partition[i][6]
        
        # Where to save the plot
        spath_m = sdir + m + '/'

        m_num_df = pd.read_csv(spath_m + 'data/num/{0}-num_{1}_data.csv'.format(str(snap), m))
        m_fit_df = pd.read_csv(spath_m + 'data/fit/{0}-fit_{1}_data.csv'.format(str(snap), m))

        centers = m_num_df['abundance'].to_list()
        num_vals = m_num_df['num_val'].to_list()
        fit_vals = m_fit_df['fit_val'].to_list()

        fig, ax = plt.subplots(figsize = (25, 15))

        if(np.min(centers) > x_min):
            empty_left = [x_min, np.min(centers)]
            zeros = [0, 0]
            ax.plot(empty_left, zeros, lw = 8, color = 'blue')

        if(np.max(centers) < x_max):
            empty_right = [np.max(centers), x_max]
            zeros = [0, 0]
            ax.plot(empty_right, zeros, lw = 8, color = 'blue')

        ax.plot(centers, num_vals, lw = 8, label = 'Simulation')
        ax.plot(centers, fit_vals, linestyle = ':', lw = 8, label = 'Fit', color = 'blue')

        ax.set_xlabel('[{}/H]'.format(m.title()), fontsize = 74)
        ax.set_ylabel(r'$p \left( \left[ {}/H \right] \right)$'.format(m.title()), fontsize = 74)
        ax.set_title('z = {}'.format(str(round(redshift, 2))), y = 1.04, fontsize = 74)

        ax.tick_params(labelsize = 74, pad = 10)
        ax.yaxis.offsetText.set_fontsize(74)

        ax.set_xlim(x_min, x_max)
        ax.set_ylim(ymin = y_min, ymax = y_max)

        plt.legend(prop={'size': 74})

        plt.savefig(savepath + '{0}-fit_{1}.png'.format(str(snap), m), bbox_inches='tight')
        plt.close()

In [4]:
# Get all fitted indices

fitted_df = pd.read_csv(sdir + 'fitted_snap_stats.csv')
fitted_indices = fitted_df['snap'].to_list()
redshifts = fitted_df['redshift'].to_list()
num_snaps = len(fitted_indices)

# Decide desired metal

m = 'fe'
spath_m = sdir + m + '/'

In [5]:
# Find universal x_lim and y_lim across all snapshots

# Initializing 

x_min = math.inf
x_max = -math.inf
y_min = math.inf
y_max = -math.inf

# Iterating across all metals and all snapshots to find universal plot bounds

for index in fitted_indices:
        
        num_df = pd.read_csv(spath_m + 'data/num/{0}-num_{1}_data.csv'.format(str(index), m))
        
        centers = num_df['abundance'].to_list()
        num_vals = num_df['num_val'].to_list()
        
        curr_x_min = np.min(centers)
        curr_x_max = np.max(centers)
        curr_y_min = np.min(num_vals)
        curr_y_max = np.max(num_vals)
        
        if(curr_x_min < x_min):
            x_min = curr_x_min
            
        if(curr_x_max > x_max):
            x_max = curr_x_max
        
        if(curr_y_min < y_min):
            y_min = curr_y_min
            
        if(curr_y_max > y_max):
            y_max = curr_y_max
            
print(x_min)
print(x_max)
print(y_min)
print(y_max)

-3.975
0.9250000000000028
0.0
54411860.0


In [6]:
# Make data arrays for each snapshot
    
if __name__ == '__main__':
    
    n_proc = multiprocessing.cpu_count()
    
    snap_infos = []

    partition_ids = []

    for i in range(0, num_snaps):

        snap_infos.append([fitted_indices[i], redshifts[i], x_min, x_max, y_min, y_max, m])
        partition_ids.append(i%n_proc)
        
    sim_partitions = []
    
    for i in range(0, n_proc):
        sim_partitions.append([])
    
    for i in range(0, num_snaps):
        sim_partitions[partition_ids[i]].append(snap_infos[i])
        
    with Pool(processes = n_proc) as pool:
        
        pool.map(plot_sim_partition, sim_partitions)

In [7]:
times = np.array(fitted_df['time'])
delta_times = np.max(times) - np.min(times)
times_range = np.arange(np.min(times), np.max(times), 0.01)

fit_params_df = pd.read_csv(spath_m + 'data/fit/fit_{}_params.csv'.format(m))

spreads = np.array(fit_params_df['std'])
delta_spreads = np.max(spreads) - np.min(spreads)
spreads_range = np.arange(np.min(spreads), np.max(spreads), 0.01)

medians = np.array(fit_params_df['median_mass'])
delta_medians = np.max(medians) - np.min(medians)
medians_range = np.arange(np.min(medians), np.max(medians), 0.01)

mach_numbers = np.array(fitted_df['mach_number_mass'])

In [8]:
def exp_fit(X, A, alpha):
    Y = A*np.exp(alpha*X)
    return Y

In [9]:
eps = 0.005

In [10]:
fig, axs = plt.subplots(nrows = 2, ncols = 1, figsize = (25, 30), sharex = True)

# Plot median

ax = axs[0]

B = np.max(medians) + eps

shift_times = times - np.min(times)
shift_medians = medians - B
shift_medians_ratio = shift_medians[len(shift_medians)-1]/shift_medians[0]

A = shift_medians[0]
alpha = np.log(shift_medians_ratio)/delta_times

guess_params = [A, alpha]

fit_params, fit_covar = optimize.curve_fit(exp_fit, shift_times, shift_medians, p0 = guess_params)
print(fit_params)

shift_times_range = np.arange(np.min(shift_times), np.max(shift_times), 0.01)
shift_fit_medians = exp_fit(shift_times_range, *fit_params)
fit_medians = shift_fit_medians + B

ax.scatter(times, medians, s = 100, label = 'Simulation', color = 'green', alpha = 0.5)
ax.plot(times_range, fit_medians, color = 'green', lw = 8, label = 'Fit')

ax.set_ylabel('Median Abundance', labelpad = 10, fontsize = 74)

ax.tick_params(labelsize = 74, pad = 10)
ax.yaxis.offsetText.set_fontsize(74)

ax.legend(prop={'size': 74})

# Plot spread

ax = axs[1]

B = np.min(spreads) - eps

shift_times = times - np.min(times)
shift_spreads = spreads - B
shift_spreads_ratio = shift_spreads[len(shift_spreads)-1]/shift_spreads[0]

A = shift_spreads[0]
alpha = np.log(shift_spreads_ratio)/delta_times

guess_params = [A, alpha]

fit_params, fit_covar = optimize.curve_fit(exp_fit, shift_times, shift_spreads, p0 = guess_params)
print(fit_params)

shift_times_range = np.arange(np.min(shift_times), np.max(shift_times), 0.01)
shift_fit_spreads = exp_fit(shift_times_range, *fit_params)
fit_spreads = shift_fit_spreads + B

ax.scatter(times, spreads, s = 100, color = 'green', alpha = 0.5)
ax.plot(times_range, fit_spreads, color = 'green', lw = 8)

ax.set_xlabel('Time (Gyr)', labelpad = 10, fontsize = 74)
ax.set_ylabel('Abundance Spread', labelpad = 72, fontsize = 74)

ax.tick_params(labelsize = 74, pad = 10)
ax.yaxis.offsetText.set_fontsize(74)

# Save file

plt.savefig(savepath + 'abundance_time.png', bbox_inches='tight')
plt.close()

[-1.27400227 -0.24430009]
[ 0.34365203 -0.24926957]


In [11]:
fig, axs = plt.subplots(nrows = 2, ncols = 1, figsize = (25, 30), sharex = True)

# Turbulence and median abundance
ax = axs[0]

B = np.min(mach_numbers) - eps

shift_medians = medians - np.min(medians)
shift_mach_numbers = mach_numbers - B
shift_mach_numbers_ratio = shift_mach_numbers[len(shift_mach_numbers)-1]/shift_mach_numbers[0]

A = shift_mach_numbers[0]
alpha = np.log(shift_mach_numbers_ratio)/delta_medians

guess_params = [A, alpha]

fit_params, fit_covar = optimize.curve_fit(exp_fit, shift_medians, shift_mach_numbers, p0 = guess_params)
print(fit_params)

shift_medians_range = np.arange(np.min(shift_medians), np.max(shift_medians), 0.01)
shift_fit_mach_numbers = exp_fit(shift_medians_range, *fit_params)
fit_mach_numbers = shift_fit_mach_numbers + B

ax.scatter(mach_numbers, medians, s = 100, label = 'Simulation', color = 'red', alpha = 0.5)
ax.plot(fit_mach_numbers, medians_range, color = 'red', lw = 8, label = 'Fit')

ax.set_ylabel('Median Abundance', labelpad = 10, fontsize = 74)

ax.tick_params(labelsize = 74, pad = 10)
ax.yaxis.offsetText.set_fontsize(74)

ax.legend(prop={'size': 74})

# Turbulence and abundance spread

ax = axs[1]

B = np.min(mach_numbers) - eps

shift_spreads = spreads - np.min(spreads)
shift_mach_numbers = mach_numbers - B
shift_mach_numbers_ratio = shift_mach_numbers[len(shift_mach_numbers)-1]/shift_mach_numbers[0]


A = shift_mach_numbers[0]
alpha = np.log(shift_mach_numbers_ratio)/delta_spreads

guess_params = [A, alpha]

fit_params, fit_covar = optimize.curve_fit(exp_fit, shift_spreads, shift_mach_numbers, p0 = guess_params)
print(fit_params)

shift_spreads_range = np.arange(np.min(shift_spreads), np.max(shift_spreads), 0.01)
shift_fit_mach_numbers = exp_fit(shift_spreads_range, *fit_params)
fit_mach_numbers = shift_fit_mach_numbers + B

ax.scatter(mach_numbers, spreads, s = 100, color = 'red', alpha = 0.5)
ax.plot(fit_mach_numbers, spreads_range, color = 'red', lw = 8)

ax.set_xlabel('Mean Mach-Number', labelpad = 10, fontsize = 74)
ax.set_ylabel('Abundance Spread', labelpad = 72, fontsize = 74)

ax.tick_params(labelsize = 74, pad = 10)
ax.yaxis.offsetText.set_fontsize(74)

plt.savefig(savepath + 'turbulence-abundance.png', bbox_inches='tight')
plt.close()

[0.04257373 3.03963119]
[  6.40703031 -11.47228288]
